In [11]:
import os

import semantic_kernel as sk
import semantic_kernel.connectors.ai.open_ai as sk_oai
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.prompt_template.input_variable import InputVariable
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.functions.kernel_arguments import KernelArguments

In [12]:
api_key, org_id = sk.openai_settings_from_dot_env()
kernel = sk.Kernel()
kernel.add_service(OpenAIChatCompletion(service_id="oai_chat_gpt", ai_model_id="gpt-3.5-turbo-1106", api_key=api_key, org_id=org_id))


In [13]:
## Function from Prompt
prompt = """
{{$user_input}}

Write an email body with the context given above.
Do not need subject, signatures or anything else. Just plain body. No other messages or anything else.
"""
execution_settings = sk_oai.OpenAIChatPromptExecutionSettings(
        service_id="oai_chat_gpt",
        ai_model_id="gpt-3.5-turbo-1106",
        max_tokens=2000,
        temperature=0.7,
    )

prompt_template_config = sk.PromptTemplateConfig(
    template=prompt,
    name="email body",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="user_input", description="The user input", is_required=True),
    ],
    execution_settings=execution_settings
)

emailfunction = kernel.create_function_from_prompt(function_name="EmailContent", plugin_name="EmailHelpers", prompt_template_config=prompt_template_config)
# email_body = await kernel.invoke(emailfunction, sk.KernelArguments(user_input="Ask everyone invited to join the call since it is time. And mention that this mail was generated using Semantic Kernel, the tech that is the main showcase of this call."))
# print(email_body)

In [14]:
sendplugin = kernel.import_native_plugin_from_directory("plugins", "EmailPlugins")
sendplugin

KernelPlugin(name='EmailPlugins', description=None, functions={'sendEmail': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='sendEmail', plugin_name='EmailPlugins', description='Send Email to the recipient with the email body', parameters=[KernelParameterMetadata(name='recipient', description='The recipient is an emailid string', default_value=None, type_='str', is_required=True, type_object=<class 'str'>), KernelParameterMetadata(name='email_body', description='The email_body is a string', default_value=None, type_='str', is_required=True, type_object=<class 'str'>)], is_prompt=False, is_asynchronous=False, return_parameter=KernelParameterMetadata(name='return', description='', default_value=None, type_='None', is_required=False, type_object=None)), method=<bound method Email.send_email of <native_function.Email object at 0x11f776c90>>, stream_method=None)})

In [16]:
await kernel.invoke(sendplugin['sendEmail'], recipient="shrashti.gupta@publicissapient.com", email_body="Check Mail Shrashti")

Error occurred while invoking function sendEmail: There may be at most 1 To headers in a message


KernelInvokeException: Error occurred while invoking function: 'EmailPlugins.sendEmail'

In [6]:
from semantic_kernel.planners.stepwise_planner import StepwisePlanner, StepwisePlannerConfig

planner = StepwisePlanner(kernel, StepwisePlannerConfig(max_iterations=10, min_iteration_time_ms=1000))

Overwriting plugin "StepwisePlanner" in collection


In [7]:
plan = planner.create_plan(goal="Write an email to Shrashti to ask when she is free to connect.")

In [8]:
result = await plan.invoke(kernel)

Error parsing XML of prompt: mismatched tag: line 74, column 64
Error parsing XML of prompt: mismatched tag: line 81, column 146


In [9]:
plan._state

{'input': 'The email has been successfully created and sent to Shrashti asking when she is free to connect.',
 'PLAN.RESULT': 'The email has been successfully created and sent to Shrashti asking when she is free to connect.'}

In [ ]:
plan

In [ ]:
for index, step in enumerate(plan._steps):
    print("Step:", index)
    print("Description:", step.description)
    print("Function:", step.plugin_name + "." + step._function.name)
    print(f"  Output: {','.join(str(res) for res in result.metadata['results'])}")

In [ ]:
## will add later
prompt = """
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$user_input}}
ChatBot: """

prompt_template_config = sk.PromptTemplateConfig(
    template=prompt,
    name="chat",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="input", description="The user input", is_required=True),
        InputVariable(name="history", description="The conversation history", is_required=True),
    ],
    execution_settings=execution_settings,
)

chat_function = kernel.create_function_from_prompt(
    function_name="chat",
    plugin_name="chatPlugin",
    prompt_template_config=prompt_template_config,
)

chat_history = ChatHistory()
chat_history.add_system_message("You are a helpful chatbot who is good at answering everything.")

async def chat(input_text: str) -> None:
    # Save new message in the context variables
    print(f"User: {input_text}")
    chat_history.add_user_message(input_text)

    # Process the user message and get an answer
    # answer = await kernel.invoke(chat_function, KernelArguments(user_input=input_text, history=chat_history))
    answer = await kernel.invoke(emailfunction, KernelArguments(user_input=input_text))

    # Show the response
    print(f"ChatBot: {answer}")

    chat_history.add_assistant_message(str(answer))
    return answer

res = await chat("Ask everyone to join the call")